## Importing Necessary Library

In [1]:
# Numerical and Data Analysis
import numpy as np
import pandas as pd

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Extra
import warnings
warnings.filterwarnings('ignore')

## Data Loading

In [2]:
df=pd.read_csv('loan.csv')

In [3]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB


In [5]:
df.shape

(39717, 111)

## Data Cleaning

### Drop Empty Columns

In [6]:
empty_columns = df.columns[df.isna().sum() == len(df)].to_list()
df1 = df.drop(columns=empty_columns)
df1.shape

(39717, 57)

### Drop Single Valued Column

In [7]:
col1 = df1.columns[df1.nunique()==1].to_list()
df2= df1.drop(columns=col1)
df2.shape

(39717, 48)

### Drop columns having more than 60% of missing values

In [8]:
check = df2.isnull().mean() * 100
cols = check[check > 60].index
df3=df2.drop(list(cols),axis=1)
df3.shape

(39717, 45)

#### There are some unnecessary columns which  donot help in analysis - id,member_id, emp_title, url, desc, title,zip_code,addr_state

In [9]:
cols = ['id','member_id', 'emp_title', 'url', 'desc', 'title','zip_code','addr_state','pub_rec_bankruptcies',
        'last_pymnt_d','last_pymnt_amnt',"last_credit_pull_d","out_prncp_inv"]
df4=df3.drop(list(cols),axis=1)
df4.shape

(39717, 32)

In [10]:
df4.isnull().mean() * 100

loan_amnt                  0.000000
funded_amnt                0.000000
funded_amnt_inv            0.000000
term                       0.000000
int_rate                   0.000000
installment                0.000000
grade                      0.000000
sub_grade                  0.000000
emp_length                 2.706650
home_ownership             0.000000
annual_inc                 0.000000
verification_status        0.000000
issue_d                    0.000000
loan_status                0.000000
purpose                    0.000000
dti                        0.000000
delinq_2yrs                0.000000
earliest_cr_line           0.000000
inq_last_6mths             0.000000
open_acc                   0.000000
pub_rec                    0.000000
revol_bal                  0.000000
revol_util                 0.125891
total_acc                  0.000000
out_prncp                  0.000000
total_pymnt                0.000000
total_pymnt_inv            0.000000
total_rec_prncp            0

## Update Missing Value 
#### There are 2 columns with missing value emp_length,revol_util

In [11]:
mod=df4['emp_length'].mode()[0]
print('Mode:',mod)

Mode: 10+ years


#### we are replacing missing values with mode value

In [12]:
df4["emp_length"]=df4["emp_length"].fillna(mod) 

#### "revol_util" column although described as an object column, it has continous values and number of missing values are very less, so we are dropping columns having missing values

In [13]:
df4.shape


(39717, 32)

In [14]:
df4.dropna(axis = 0, subset = ['revol_util'] , inplace = True)
df4.revol_util.isna().sum()
df4.shape

(39667, 32)

In [15]:
df4.isnull().mean() * 100

loan_amnt                  0.0
funded_amnt                0.0
funded_amnt_inv            0.0
term                       0.0
int_rate                   0.0
installment                0.0
grade                      0.0
sub_grade                  0.0
emp_length                 0.0
home_ownership             0.0
annual_inc                 0.0
verification_status        0.0
issue_d                    0.0
loan_status                0.0
purpose                    0.0
dti                        0.0
delinq_2yrs                0.0
earliest_cr_line           0.0
inq_last_6mths             0.0
open_acc                   0.0
pub_rec                    0.0
revol_bal                  0.0
revol_util                 0.0
total_acc                  0.0
out_prncp                  0.0
total_pymnt                0.0
total_pymnt_inv            0.0
total_rec_prncp            0.0
total_rec_int              0.0
total_rec_late_fee         0.0
recoveries                 0.0
collection_recovery_fee    0.0
dtype: f

### Standardizing the data

In [16]:
df4.dtypes

loan_amnt                    int64
funded_amnt                  int64
funded_amnt_inv            float64
term                        object
int_rate                    object
installment                float64
grade                       object
sub_grade                   object
emp_length                  object
home_ownership              object
annual_inc                 float64
verification_status         object
issue_d                     object
loan_status                 object
purpose                     object
dti                        float64
delinq_2yrs                  int64
earliest_cr_line            object
inq_last_6mths               int64
open_acc                     int64
pub_rec                      int64
revol_bal                    int64
revol_util                  object
total_acc                    int64
out_prncp                  float64
total_pymnt                float64
total_pymnt_inv            float64
total_rec_prncp            float64
total_rec_int       

#### Convert columns having percentages as interests data type from object to floating type

In [17]:
print(df4[['int_rate', 'revol_util']].head())


  int_rate revol_util
0   10.65%     83.70%
1   15.27%      9.40%
2   15.96%     98.50%
3   13.49%        21%
4   12.69%     53.90%


In [18]:
df4['int_rate'] = df4['int_rate'].str.replace('%', '').astype(float)
df4['revol_util'] = df4['revol_util'].str.replace('%', '').astype(float)

In [19]:
print(df4[['int_rate', 'revol_util']].head())

   int_rate  revol_util
0     10.65        83.7
1     15.27         9.4
2     15.96        98.5
3     13.49        21.0
4     12.69        53.9


### "emp_length" --> { (< 1 year) is assumed as 0 and 10+ years is assumed as 10 }, so we are changing the vlaues accordingly

In [20]:
df4['emp_length'] =df4['emp_length'].apply(lambda x: 0 if "<" in x else (x.split('+')[0] if "+" in x else x.split()[0])).astype(int)
print(df4['emp_length'])

0        10
1         0
2        10
3        10
4         1
         ..
39712     4
39713     3
39714     0
39715     0
39716     0
Name: emp_length, Length: 39667, dtype: int32


In [21]:
df4.dtypes

loan_amnt                    int64
funded_amnt                  int64
funded_amnt_inv            float64
term                        object
int_rate                   float64
installment                float64
grade                       object
sub_grade                   object
emp_length                   int32
home_ownership              object
annual_inc                 float64
verification_status         object
issue_d                     object
loan_status                 object
purpose                     object
dti                        float64
delinq_2yrs                  int64
earliest_cr_line            object
inq_last_6mths               int64
open_acc                     int64
pub_rec                      int64
revol_bal                    int64
revol_util                 float64
total_acc                    int64
out_prncp                  float64
total_pymnt                float64
total_pymnt_inv            float64
total_rec_prncp            float64
total_rec_int       

### Converting issue_d and earliest_cr_line to datetime data type

In [ ]:
df4['issue_d'] = df4['issue_d'].apply(lambda x: pd.to_datetime(x, format='%b-%y'))
df4['earliest_cr_line'] = df4['earliest_cr_line'].apply(lambda x: pd.to_datetime(x, format='%b-%y'))

In [ ]:
df4.nunique()

#### term, grade, sub_grade,verification_status, loan_status,purpose as cteagorical column so consider these as object type

## Dropping data which are not relevant


In [ ]:
print(df4['loan_status'].unique())

## The purpose of the analysis is to see who is likely to default and this can only be said in case of either fully paid or charged off loans.
### We cannot make anything up for the current loans.
### To exclude that data , removing the records with current loan status

In [ ]:
final_data = df4[df4['loan_status'] != "Current"]
final_data['loan_status'].unique()

In [ ]:
final_data.shape

In [ ]:
final_data.nunique()

### Handling Outliners
##### We can analyze on 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'installment', 'annual_inc','dti',, 'revol_bal', 'revol_util' columns as we found these important for analysis

In [ ]:
columns_to_plot = ['annual_inc','loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'installment', 'dti', 'revol_bal', 'revol_util']

for col_name in columns_to_plot:
    plt.figure(figsize=(2, 2))
    sns.boxplot(y=final_data[col_name])
    plt.show()

### These plot clearly represents presence of some outliners so removing them. 



In [ ]:
quantile_info = final_data['annual_inc'].quantile([0.5, 0.75,0.90, 0.95, 0.97,0.98, 0.99])
quantile_info

In [ ]:
quant_95_value= final_data['annual_inc'].quantile(0.95)
final_data = final_data[final_data.annual_inc <= quant_95_value]
final_data.shape

#### for annual_inc considering only till 95 percentile as values after this seems to be outliners.

In [ ]:
plt.figure(figsize=(5, 4))
sns.boxplot(x=final_data['annual_inc'])
plt.show()

In [ ]:
quantile_info = final_data['loan_amnt'].quantile([0.5, 0.75,0.90, 0.95, 0.97,0.98, 0.99])
quantile_info

In [ ]:
quantile_info = final_data['funded_amnt'].quantile([0.5, 0.75,0.90, 0.95, 0.97,0.98, 0.99])
quantile_info

## Arranging the fields in Categorical, Numerical and Date variables
### Unordered Categorical Variables
- term - 36 or 60 months
- grade - LC assigned loan grade
- sub_grade - it is sub-division of grade
- home_ownership - home ownership status RENT, OWN, MORTGAGE, OTHER.
- verification_status - Indicates if income was verified by LC, not verified, or if the income source was verified
- loan_status
- purpose

### Ordered Categorical Variables
- emp_length - Employment length in years
- inq_last_6mths - The number of inquiries in past 6 months

## Quantitative Variables
- loan_amnt
- funded_amnt
- funded_amnt_inv - This is almost same as 'funded_amnt' field
- int_rate
- installment
- annual_inc
- dti 
- delinq_2yrs
- pub_rec
- total_rec_prncp- Principal received to date
- revol_bal
- revol_util 
- total_acc
- open_acc

## Date Variables
- issue_d
- earliest_cr_line

# Univariate Analysis

In [ ]:
# This function sums the total by the categorical variable we are analyzing
def bycategory(x):
    ax = sns.countplot(x=final_data[x], order = final_data[x].value_counts().index)
    for i in ax.containers:
        ax.bar_label(i,)
    ax.tick_params(axis='x', rotation=90)
    return ax

In [ ]:
# This function plots percentages by loan status 
def byloanstatuspercent(x):
    cross_tab_prop = pd.crosstab(index=final_data[x],
                             columns=final_data['loan_status'],
                             normalize="index")
    ax = cross_tab_prop.plot(kind='bar', 
                    stacked=True, 
                    colormap='tab10', 
                    figsize=(10, 5))

    plt.legend( ncol=2)
    plt.xlabel(x)
    plt.ylabel("% of Total")
    plt.xticks(rotation=0)
    
    
    
    
    for n, x in enumerate([*cross_tab_prop.index.values]):
        for proportion in cross_tab_prop.loc[x]:
            plt.text(x=n,
                 y=proportion,
                 s=f'{np.round(proportion * 100, 1)}%', 
                 color="black",
                 fontsize=12,
                 fontweight="bold")
    
    return ax
                
  

## Term

In [ ]:
bycategory('term')
byloanstatuspercent('term')
plt.show()

**Observations:**

* 36 months term loans are more common than 60 months
* 60 months term loans are more likely to default

## Grade

In [ ]:
bycategory('grade')
byloanstatuspercent('grade')
plt.show()

**Observations:**
* more than 50% of loans are of high grade, A or B
* Grade and chances of default are inversely proportaional. We can say Lending Club is quite good at grading it's loans

## Home Ownership

In [ ]:
bycategory('home_ownership')
byloanstatuspercent('home_ownership')
plt.show()

**Observations:**
* Most of the loans are by people on rent or mortgage
* There is no significant impact of home ownership on default

## Verification

In [ ]:
bycategory('verification_status')
byloanstatuspercent('verification_status')
plt.show()

**Observations:**
* Almost 50% of the loans are not verified
* The chances of default for verified loans are more than not verified loans which is quite counter-intuitive

## pub_rec

In [ ]:
bycategory('pub_rec')
byloanstatuspercent('pub_rec')
plt.show() 

**Observations:**
* Chances of default increases significantly when borrower has prior derogatory public record

## Employee Length

In [ ]:
bycategory('emp_length')

byloanstatuspercent('emp_length')

**Observations:**
* There is no significant impact of employee length on loan default

## Purpose

In [ ]:
bycategory('purpose')
plt.show()
byloanstatuspercent('purpose')
plt.xticks(rotation=90)
plt.show()

**Observations:**
* Debt consolidation comprises of almost 49% of the loans, followed by credit card
* 27.1% of loans to small businesses are defaulted, making them the most risky ones

## inq_last_6mths

In [ ]:
bycategory('inq_last_6mths')

byloanstatuspercent('inq_last_6mths')
plt.show()

**Observations:**
* Highest number of loans never had any inquiry and are less likely to default
* Loans with more than 2 inquiries has higher rate of default

## delinq_2yrs

In [ ]:
bycategory('delinq_2yrs')

byloanstatuspercent('delinq_2yrs')
plt.show()

**Observations:**
* Loans with more than 1 incidences of delinquency has higher rate of default

# Numerical Variables

In [ ]:
final_data.select_dtypes(['int64','float64']).columns

## Loan Amount

In [ ]:
def plotnum(x):
    plt.figure(figsize=(15,5))
    plt.subplot(1, 2, 1)
    ax = sns.distplot(final_data[x])
    plt.subplot(1, 2, 2)
    sns.boxplot(x=final_data[x], y=final_data['loan_status'], data=final_data)
    plt.show()

In [ ]:
plotnum('loan_amnt')

In [ ]:
final_data.groupby('loan_status')['loan_amnt'].describe()

**Observations:**
* The loan amount is right-skewed, where the median is less than the mean
* Defaulted loans are slightly on the higher side of the average loan amount

## Interest Rate

In [ ]:
plotnum('int_rate')
final_data.groupby('loan_status')['int_rate'].describe()

**Observations:**
* Interest rates for defaulted loans are higher than fully paid ones. One inference can be that they are more likely to be the riskier loans

## Installment

In [ ]:
plotnum('installment')
final_data.groupby('loan_status')['installment'].describe()

**Observations:**
* Defaulted loans are likely to have slightly more installments then fully paid ones

## DTI

In [ ]:
plotnum('dti')
final_data.groupby('loan_status')['dti'].describe()

In [ ]:
plt.figure(figsize=(10,5))
bins = [0, 5, 10, 15, 20,30]
final_data['dti_groups'] = pd.cut(final_data['dti'], bins=bins,precision =0,labels=['0-5','5-10','10-15','15-20','20+'])

In [ ]:
final_data[['dti','dti_groups']].head(25)

In [ ]:
bycategory('dti_groups')
byloanstatuspercent('dti_groups')
plt.show()

**Observations:**
* Higher the dti higher the chances of loan being Charged Off

## Open Account

In [ ]:
plotnum('open_acc')
final_data.groupby('loan_status')['open_acc'].describe()

**Observations:**
* No significant impact of open account on loan default

## Revolving Balance

In [ ]:
plotnum('revol_bal')
final_data.groupby('loan_status')['revol_bal'].describe()

**Observations:**
* No significant impact of revolving balance on loan default

## Revolving Utilization

In [ ]:
plotnum('revol_util')
final_data.groupby('loan_status')['revol_util'].describe()

## Total Account

In [ ]:
plotnum('total_acc')
final_data.groupby('loan_status')['total_acc'].describe()

## Total Payment

In [ ]:
plotnum('total_pymnt')
final_data.groupby('loan_status')['total_pymnt'].describe()

**Observations:**
* the variable will not be available at the time of funding, hence, can be dropped

## Total Payment Invested

In [ ]:
plotnum('total_pymnt_inv')
final_data.groupby('loan_status')['total_pymnt_inv'].describe()

**Observations:**
* the variable will not be available at the time of funding, hence, can be dropped

## total_rec_int

In [ ]:
plotnum('total_rec_int')
final_data.groupby('loan_status')['total_rec_int'].describe()

**Observations:**
* the variable will not be available at the time of funding, hence, can be dropped

## Annual Income

In [ ]:
plt.figure(figsize=(10,5))
bins = [0, 25000, 50000, 75000, 100000,150000]
final_data['annual_inc_groups'] = pd.cut(final_data['annual_inc'], bins=bins,precision =0,labels=['0-25000','25000-50000','50000-75000','75000-100000','100000+'])

In [ ]:
final_data[['annual_inc','annual_inc_groups']].head(25)

In [ ]:
bycategory('annual_inc_groups')
byloanstatuspercent('annual_inc_groups')
plt.show()

**Observations:**
* Borrowers with income less than 25K are more likely to default
* Loan default decreases with higher annual income

## Bivariant Analysis

In [ ]:
plt.figure(figsize=(4, 4))
plt.title("Loan Status Vs Loan Amount")
sns.boxplot(x=final_data["loan_status"],y=final_data["loan_amnt"])
plt.show()

**Observations:**
* Chances of getting loan amount charged off is more than fully paid.

In [ ]:
plt.figure(figsize=(4, 4))
plt.title("Funded Amount Vs Loan Amount")    
sns.scatterplot(x=final_data['loan_amnt'],y=final_data['funded_amnt'])
plt.show()

**Observations:**
* Funded amount by the company is always less than or equal to loan amount applied by borrower

In [ ]:
plt.figure(figsize=(4, 4))
plt.title("Funded Amount Vs Installment")    
sns.scatterplot(x=final_data['installment'],y=final_data['funded_amnt'])
plt.show()

**Observations:**
* Funded amount by the company is highly correlated with monthly installment paid by the borrower. 
* By this we can conclude there is a high correlation between loan amount, funded amount and installments


In [ ]:
plt.figure(figsize=(10,5))
final_data['loan_amnt_groups'] = pd.cut(final_data['loan_amnt'], bins=6,precision =0,labels=['0-5000','5000-10000','10000-15000','15000-20000','20000-25000','25000-30000'])
sns.barplot(data =final_data,x='loan_amnt_groups', y='int_rate', hue ='loan_status')
plt.show()

**Observations:**
* There is a more probability of defaulting when applicants who have taken a loan in the range 30k - 35k and are charged interest rate of 15-17.5 %

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(221)
sns.barplot(data =final_data,y='loan_amnt', x='emp_length', hue ='loan_status')

**Observations:**
* Employees with tenure as 10 years usually takes loans of higher amount and the risk is also low as compared to other tenure

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(221)
sns.barplot(data =final_data,y='loan_amnt', x='home_ownership', hue ='loan_status')

**Observations:**
* Loans for Individuals living on rent usually have a higher chance of getting defaulted as compared to others. Applicants whose home ownership is 'MORTGAGE are having more probability for getting loan charged off

## Multivariant Analysis

#### Buisness Driven Column 
#####  Convert loan_status to 0 for "Fully Paid" and 1 for "Charged Off"

In [ ]:
final_data['loan_status_numeric'] = final_data['loan_status'].apply(lambda x: 1 if x == 'Charged Off' else 0)

In [ ]:
final_data_numeric = final_data.select_dtypes(include=['int64', 'float64'])
corr_matrix = final_data_numeric.corr()
plt.figure(figsize=(12, 12))
heatmap = sns.heatmap(corr_matrix, annot=True, cmap='BuGn', fmt=".2f")
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12})
plt.show()

**Observations:**
* Looks like higher int_rate (0.21) can lead to high defaulters
* Higher recoveries (0.20) can lead to high defaulters
* Higher collection_recovery_fee (0.20) can lead to high defaulters
* Positive correlation can also be found on funded_amnt, funded_amnt_inv, installment, dti, delinq_2yrs, inq_last_6mnths, pub_rec, revol_bal, revol_util, total_rec_int, total_rec_late_fee, pub_rec_bankruptcies
* Very high correlation between loan amount applied by the borrower and funded amount committed by the investor. We can say that if loan application of the borrower gets passed by company, then he gets almost full amount of the loan applied.
* Certainly loan amount, monthly installment and principal received are highly correlated. Higher the loan amount, higher is monthly installment and and higher is principal received  vice versa.
* Number of open credit lines is also highly correlated with total number of credit lines in the borrower's credit file.  
* Revol_util, int_rate - Moderately high correlation
* Higher the amount of credit the borrower is using, higher is the risk of loan repayment. Thats why higher is the interest rate, and vice-versa.
